In [ ]:
import ModelingToolkit as Model
import SymPy as sp
import Symbolics as Symb
using DomainSets
import ApproxFun as AF
import DifferentialEquations as DE
using CairoMakie
using GLMakie
include("multiharmonic_balance.jl");

# Harmonic Balance using multiple harmonics

## Use the SymPy method to start a potential Harmonic Balance

$$
u = \sum_{k=1}^H A_k \cos(k\omega t) + B_k \sin(k\omega t)
$$

Define the variable parameters for the wave equation

In [ ]:
gamma = 0;
omega = 3;
gamma3 = 0;
g0 = 9.80665; # m / s^2
height = 5; # m
harmonics = 5; # number of harmonics

Define the constants specific to the discretizations

In [ ]:
xleft::Float64 = 0.0;
xright::Float64 = 1.0;
Nt = 5
N = 100;
order = 2;
stepx = (xright-xleft)/N;

In [ ]:
# Define symbolics
Model.@parameters x, t;

Dx = Model.Differential(x);
Dt = Model.Differential(t);

In [ ]:
u0 = ones((N - 1) * harmonics * 2);

In [ ]:
vars, var_exprs, u = create_ansatz((x,), t, omega, harmonics);
#bcs = create_bcs(vars, ((xleft, xright),), (x,), 0.0)

F = sin(omega*t)# 50 * exp(-40*(x^2))*sin(omega*t)

y = Dt(Dt(u)) - 9*Dx(Dx(u)) + gamma*Dt(u) + gamma3*Dt(u)*Dt(u)*Dt(u) - F;

Model.@variables x, t;
expanded = expand_trig(y, t, omega);

eqs = make_residual(expanded, harmonics, omega, t);


In [ ]:
u

In [ ]:
eqs

In [ ]:
using MacroTools: @capture, postwalk, prewalk

In [ ]:
function transform_sym(ex)::Expr
    prewalk(ex) do x
        if @capture(x, Differential(x)(Differential(x)(s_(x))))
            return :(($s[i+1]-2* $s[i]+$s[i-1])/dx^2)
        elseif @capture(x, Differential(x)(s_(x)))
            return :(($s[i+1] - $s[i-1]) / (2*dx))
        elseif @capture(x, s_(x))
            return :($s[i])
        end
        x
    end
end

In [ ]:
sym_eqs = map(transform_sym ∘ Meta.parse ∘ string, eqs)

In [ ]:
function create_residual_function_1D(N::Int64, harmonics::Int64, eqs::Vector{Expr})
    Us = [Symbol(Char('A' + i - 1)) for i in 1:2*harmonics]
    Fs = [Symbol(string("F_", Char('A' + i - 1))) for i in 1:2*harmonics]
    quote
        function residual!(F, U, p)
            dx = p

            $([:($v = U[($i-1)*$N+$i:$i*$N+$i]) for (i, v) in enumerate(Us)]...)
            $([:($v = F[($i-1)*$N+$i:$i*$N+$i]) for (i, v) in enumerate(Fs)]...)

            for i in 2:$(N)
                $([:(($v)[i] = $(eqs[j])) for (j, v) in enumerate(Fs)]...)
            end

            return F
        end
    end
end

In [ ]:
create_residual_function_1D(N, harmonics, sym_eqs)